# Validation of job titles in bios

In [1]:
# Standard library imports
import sys

# Third party imports
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import seaborn as sns
import numpy as np
import prince
from networkx.algorithms import bipartite
from matplotlib.patches import Patch
import community as community_louvain
from netgraph import Graph


from nltk import ngrams
from collections import Counter

# Local application imports
sys.path.insert(0, '../Utility files')


/tmp/ipykernel_1515732/2404566381.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:

# load data
path = '/home/livtollanes/NewData/french/french1_updated.csv'

req_cols = ['follower_id', 'screen_name', 'description','description_cleantext','location', 'language', 'country']
dtypes = {'follower_id': 'object',
          'screen_name': 'object', 
          'description': 'object',
          'description_cleantext': 'object',
          'location': 'object',
          'language': 'object',
          'country': 'object'}


bios = pd.read_csv(path, usecols=req_cols, dtype=dtypes)

In [3]:
bios.head()

,follower_id,screen_name,description,location,description_cleantext,language,country
0,2419429508,Max_Mth_,"Radio, web, magazine - culture, société, vie a...","paris, france","Radio, web, magazine - culture, société, vie a...",fr,France
1,902419434358403072,Hellodie41,Ma thérapie : raconter ma vie a des gens qui n...,"blois, france",Ma thérapie : raconter ma vie a des gens qui n...,fr,France
2,44372703,julienmielcarek,Directeur des rédactions digitales BFM-RMC / C...,paris,Directeur des rédactions digitales BFM-RMC / C...,fr,France
3,1640041946,QDebrand,Directeur Adjoint @LDLC_Arena,"lyon, france",Directeur Adjoint @LDLC_Arena,fr,France
4,3118973117,jeanluc_bouchet,cadre informatique secteur hospitalier\nIntérê...,"betton, france",cadre informatique secteur hospitalierIntérêt ...,fr,France


the data below comes from https://www.insee.fr/fr/information/6450559

In [4]:
titles = pd.read_excel('/home/livtollanes/10.jan-thesis/labeldata/titles.xlsx')

In [5]:
titles.head()

,title_man,title_female
0,Abat-jouriste,Abat-jouriste
1,Abatteur d'arbres,Abatteuse d'arbres
2,Abatteur de bestiaux,Abatteuse de bestiaux
3,Abatteur de carrières,Abatteuse de carrières
4,Abatteur de mines,Abatteuse de mines


In [7]:
words1 = titles['title_female'].tolist()
words2 = titles['title_man'].tolist()

# Convert all items in the list to strings
words1 = [str(item) for item in words1]
words2 = [str(item) for item in words2]

# Create a pattern for the words to be searched
pattern1 = '|'.join(words1)
pattern2 = '|'.join(words2)

# Use str.contains to get rows where 'description_cleantext' contains any word from the lists
filtered_df1 = bios[bios['description_cleantext'].str.contains(pattern1, na=False)]
filtered_df2 = bios[bios['description_cleantext'].str.contains(pattern2, na=False)]

/tmp/ipykernel_1515732/841559411.py:13: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df1 = bios[bios['description_cleantext'].str.contains(pattern1, na=False)]
/tmp/ipykernel_1515732/841559411.py:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df2 = bios[bios['description_cleantext'].str.contains(pattern2, na=False)]


In [8]:
import re

# Define a function that finds matching words in a string for words1
def find_matching_words1(description):
    matches = []
    for word in words1:
        # Split the word by commas and parentheses
        parts = re.split(',|\(|\)', word)
        # Check each part separately
        for part in parts:
            part = part.strip()  # Remove leading and trailing spaces
            if part and part in description:
                matches.append(part)
    return matches

# Apply the function to the 'description_cleantext' column and assign the result to a new column 'job1'
filtered_df1['job1'] = filtered_df1['description_cleantext'].apply(find_matching_words1)

# Define a function that finds matching words in a string for words2
def find_matching_words2(description):
    matches = []
    for word in words2:
        # Split the word by commas and parentheses
        parts = re.split(',|\(|\)', word)
        # Check each part separately
        for part in parts:
            part = part.strip()  # Remove leading and trailing spaces
            if part and part in description:
                matches.append(part)
    return matches

# Apply the function to the 'description_cleantext' column and assign the result to a new column 'job2'
filtered_df2['job2'] = filtered_df2['description_cleantext'].apply(find_matching_words2)

/tmp/ipykernel_1515732/1780242890.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df1['job1'] = filtered_df1['description_cleantext'].apply(find_matching_words1)
/tmp/ipykernel_1515732/1780242890.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df2['job2'] = filtered_df2['description_cleantext'].apply(find_matching_words2)


In [20]:
# Apply the function to the 'description_cleantext' column and assign the result to a new column 'job2'
filtered_df1['job1'] = filtered_df2['description_cleantext'].apply(find_matching_words1)

/tmp/ipykernel_1515732/4228196134.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df1['job1'] = filtered_df2['description_cleantext'].apply(find_matching_words1)


In [22]:
pd.set_option('display.max_colwidth', None)
filtered_df2[['description_cleantext', 'job2']].iloc[150:190]

,description_cleantext,job2
985,Suppléante du Député @T_Cazenave sur la #Gironde #Bordeaux #Bruges #LeBouscat #renaissance Responsable centre de formation,"[eau, eau, eau, Député, eau]"
986,"Maire adjointe @mairieivry en charge de la #culture, du #patrimoine, du #tourisme, de la #villenumérique, des #travaux et des affaires civiles et juridiques","[numérique, numérique, adjoint, numérique, Maire, Maire adjoint, Maire, Maire, adjoint, numérique, numérique, culture]"
999,Infirmier Libéral--NBA— #MileHighBasketball CinéSeries--Numerique--Design--Lyon mais Bressan,"[Infirmier, Lyon]"
1001,"La Confédération Nationale des Avocats, 1er syndicat historique d'avocats, regroupe tous les avocats soucieux des intérêts de la profession comme de la justice.",[Avocat]
1006,Chasseuse de tête - Consultante chez @CadrAvenir / ex-Journaliste @RTL et @LCI-TF1,[Journaliste]
1020,"Avocat. Je suis responsable de ce que je dis, pas de ce vous comprenez #Teamjuriste #Babylawyer",[Avocat]
1031,Journaliste @MidiLibrelozere Localier de métier. Breton de naissance. Napolitain d'adoption. Papa de Matei #ForzaNapoliSempre #СлаваУкраїні,[Journaliste]
1036,Président de IDA (Intégrer Développer Agir) Secrétaire général de APSM (Alliance Patriotique pour la Souveraineté du Mali) président d’Espérance Média MaliDjigu,"[Secrétaire, Secrétaire général, Secrétaire général]"
1038,Avocat en droit du travail. Du côté de la force.,[Avocat]
1044,Ancienne Députée de Loire-Atlantique Circo4404 Commission Aff. Étrangères I Assemblée Parlementaire de la #Francophonie I Droits de l’Homme I #JOP2024,"[Député, Parlementaire]"


In [15]:
# Create a mask for rows where 'job2' contains only one unique value
mask = filtered_df2['job2'].apply(lambda x: len(set(x)) == 1)

# Use the mask to filter the DataFrame
unique_values_df = filtered_df2[mask]

# Print the unique values
print(unique_values_df['job2'])

11                [Psychologue]
17                [Journaliste]
34                     [Député]
50                [Journaliste]
91                [Journaliste]
                  ...          
115020      [Rédacteur en chef]
115029                 [Député]
115030                [Juriste]
115056            [Chroniqueur]
115065    [Maire, Maire, Maire]
Name: job2, Length: 6527, dtype: object


In [17]:
# Get the counts of each unique value in 'job2'
counts = unique_values_df['job2'].value_counts()

# Print the counts
pd.set_option('display.max_rows', None)
print(counts)

job2
[Journaliste]                                     2777
[Maire, Maire, Maire]                              328
[Avocat]                                           244
[Photographe]                                      231
[Juriste]                                          192
[Député]                                           126
[Secrétaire]                                       115
[Acteur]                                           110
[Comédien]                                          92
[Rédacteur en chef]                                 86
[Graphiste]                                         77
[Écrivain]                                          58
[Infirmier]                                         52
[Blogueur]                                          47
[Responsable communication]                         47
[Musicien]                                          47
[Ambassadeur]                                       44
[Chroniqueur]                                       41
[Phar